In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim import Adam
from torchvision import datasets
from torchvision import models
from torchvision.models import DenseNet169_Weights

import random

from colorama import Fore, Style

In [ ]:
# colorama
red = Fore.RED
green = Fore.GREEN
blue = Fore.BLUE
yellow = Fore.YELLOW
cyan = Fore.CYAN

reset = Style.RESET_ALL

In [ ]:
# Data
d = ".../Rocks/"

loc = "PyDL_C"

# Sub-Categorized data
train_dir = d + loc + "/data/train"
test_dir = d + loc + "/data/test"
valid_dir = d + loc + "/data/validation"

In [ ]:
# Setting the seed
seed = 42
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

print(f'{blue}Global seed set to : {yellow}{seed}\n')

In [ ]:
# Data transformations training set
transform_train = transforms.Compose([
    transforms.RandomResizedCrop((256, 256)),
    transforms.ToTensor()
])

# Data transformations for validation and test sets
transform_common = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Image dataset
dataset_train = datasets.ImageFolder(root=train_dir, transform=transform_train)
dataset_test = datasets.ImageFolder(root=test_dir, transform=transform_common)
dataset_valid = datasets.ImageFolder(root=valid_dir, transform=transform_common)

In [ ]:
# Hyperparameters
max_epoch = 50
batch_size = 16
learning_rate = [0.001, 0.0001, 1e-05]
weight_decay = [1e-07, 1e-08, 1e-09]

In [ ]:
# Data loaders
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, shuffle=False)

In [ ]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'{blue}Device: {yellow}{device}{reset}')

In [ ]:
grid_search = []

for lr in learning_rate:
    for wd in weight_decay:
        
        # ---------------------------------------------------------------------------    
        # DenseNet
        model = models.densenet169(weights=DenseNet169_Weights.IMAGENET1K_V1)

        num_classes = len(dataset_train.classes)

        # change the last layer of the model
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(in_features=num_ftrs, out_features=num_classes)
        
        model.to(device)
        
        # ---------------------------------------------------------------------------
        
        # Loss
        criterion = torch.nn.CrossEntropyLoss()
        
        # Optimizer
        optimizer = Adam(model.parameters(), lr=lr, weight_decay=wd)
        
        # ---------------------------------------------------------------------------
        
        # best model score
        max_score = 0
        
        print(f'{blue}Training started for :- {yellow}Learning Rate : {lr} | Weight Decay : {wd}{reset}\n')
        
        for epoch in range(max_epoch):
            model.train()
        
            # Metrics initialization
            running_loss = 0.0
            num_correct = 0
        
            # TRAINING
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
        
                # Zero the parameter gradients
                optimizer.zero_grad()
        
                # Predictions | forward pass | OUTPUT
                outputs = model(inputs)
                # Loss | backward pass | GRADIENT
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
        
                # Metrics
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                # Count correct predictions
                num_correct += (predicted == labels).sum().item()
        
            # ---------------------------------------------------------------------------
            # Training loss
            train_lss = running_loss / len(train_loader)
        
            # Training accuracy
            train_accuracy = 100 * num_correct / len(train_loader.dataset)
            # ---------------------------------------------------------------------------
        
            model.eval()
            correct = 0
            valid_loss = 0
        
            # VALIDATION
            with torch.no_grad():
                for data in valid_loader:
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
        
                    # Predictions
                    outputs = model(inputs)
                    # Count correct predictions
                    _, predicted = torch.max(outputs, 1)
                    correct += (predicted == labels).sum().item()
                    # Loss
                    valid_loss += criterion(outputs, labels).item()
        
            # --------------------------------------------------------------------------
            #Validation loss
            val_lss = valid_loss / len(valid_loader)
        
            # Validation accuracy
            val_accuracy = 100 * correct / len(valid_loader.dataset)
            # --------------------------------------------------------------------------
        
            if epoch % 5 == 0:
                print(f"{cyan}EPOCH {epoch+1}{reset}\t Loss: {red}{train_lss}{reset}\t Validation Accuracy: {red}{val_accuracy}%{reset}\t Training Accuracy: {red}{train_accuracy}%")
            
            # Save the best model
            if val_accuracy > max_score:
                max_score = val_accuracy
                name = f'T__{lr}__{wd}.pth'
                path = d + loc + '/models/' + name
                torch.save(model.state_dict(), path)
        
        print(f'{yellow}Training finished for : {lr} | {wd}{reset}\n\n')
        
        # Results saved
        ll = [lr, wd, max_score]
        grid_search.append(ll)
        
        # Save the Final model
        name = f'F__{lr}__{wd}.pth'
        path = d + loc + '/models/' + name
        torch.save(model.state_dict(), path)

In [ ]:
# Final models results
k = 0

for lr in learning_rate:
    for wd in weight_decay:        
        # --------------------------------------------------------------------------
        # TESTING on BEST Model
        name_T = f'T__{lr}__{wd}.pth'
        
        b_model = models.densenet169(weights=DenseNet169_Weights.IMAGENET1K_V1)
        num_classes = len(dataset_train.classes)
        num_ftrs = b_model.classifier.in_features
        b_model.classifier = nn.Linear(in_features=num_ftrs, out_features=num_classes)
        b_model.load_state_dict(torch.load(d + loc + '/models/' + name_T))        
        
        total = 0      
        correct = 0      
        
        b_model.eval()
        b_model.to(device)
        
        with torch.no_grad():
            for i, data in enumerate(test_loader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
        
                # Best Model
                # Predictions | forward pass | OUTPUT
                outputs = b_model(inputs)
                # Count correct predictions
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                    
        grid_search[k].append(100 * correct / total)
        
        # --------------------------------------------------------------------------
        # Testing on FINAL Model
        name_F = f'F__{lr}__{wd}.pth'
        
        b_model = models.densenet169(weights=DenseNet169_Weights.IMAGENET1K_V1)
        num_ftrs = b_model.classifier.in_features
        b_model.classifier = nn.Linear(in_features=num_ftrs, out_features=num_classes)
        b_model.load_state_dict(torch.load(d + loc + '/models/' + name_F))        
        
        correct = 0
        total = 0
        
        b_model.eval()
        b_model.to(device)
        
        with torch.no_grad():
            for i, data in enumerate(test_loader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
        
                # Predictions | forward pass | OUTPUT
                outputs = b_model(inputs)
                # Count correct predictions
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        grid_search[k].append(100 * correct / total)
        
        # next in grid_search
        k += 1

In [ ]:
# Results
print(f'{blue}Grid Search Results{reset}\n')

best_result = 0
best = []

for i in range(len(grid_search)):
    print(f'Learning Rate: {grid_search[i][0]} | Weight Decay: {grid_search[i][1]} | Validation Accuracy: {grid_search[i][2]} || Test : Best Model: {grid_search[i][3]} & Final Model: {grid_search[i][4]}')
    
    if grid_search[i][3] > best_result or grid_search[i][4] > best_result:
        best_result = grid_search[i][3]
        best = grid_search[i]
       
print(f'\n\n{red}Best Result : lr = {best[0]} | wd = {best[1]} | val_acc = {best[2]} || Test : Best Model: {best[3]} & Final Model: {best[4]}{reset}')